In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

from matplotlib import cm
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase
from matplotlib.ticker import FuncFormatter
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors

sys.path.append('../src')
from dataset_utils import loadlocal_mnist, loadlocal_mnist_classes, generate_gaussians
from math_utils import pca
from plot_utils import plot_mean_images, plot_reference_grid
from plot_utils import plot_blind, plot_ground_truth
from plot_utils import plot_attraction_magnitude, plot_attraction_directions, plot_polar_colormap, plot_attraction_vectors, plot_repulsion_vectors 
from tsne_utils import compute_attraction_directions

from constants import TSNE_PATH
sys.path.append(TSNE_PATH)
from fast_tsne import fast_tsne

In [ ]:
# Toy dataset of two 1000-dim gaussians with means 
# mu_1 = 3e_1 and mu_2 = 3e_2, unit covariance.

ndims = 1000
mu1 = np.zeros(ndims)
mu1[0] = 3
mu2 = np.zeros(ndims)
mu2[1] = 3

mus = [mu1, mu2]
covs = [np.eye(ndims), np.eye(ndims)]
X, lab = generate_gaussians(mus, covs, n=6000)
X = pca(X, n_dims=50)

In [ ]:
# Most of the t-SNE embeddings in the force colorings
# section of the paper were generated using these
# parameters. The choice of [sigma=1e10] came from some of
# our earlier experiments, since we were thinking about
# the structure of P and this sigma makes P kind of look
# like an adjacency matrix. The exact values of the 
# parameters should not limit the potential usefulness 
# of force colorings for analyzing force-based embeddings.

# It is important to set [load_affinities='save'], as the
# functions for plotting force colorings assume that the
# matrix P is saved in the current working directory. 

FRAC = 0.01
K=int(FRAC * X.shape[0])
%time Y = fast_tsne(X, early_exag_coeff=12, K=K, sigma=1e10, load_affinities='save')

In [ ]:
# Plot the t-SNE embedding. See [src/plot_utils/plots.py].

plot_blind(Y, suptitle='unlabeled embedding')
plot_ground_truth(Y, lab, cmap='rainbow', suptitle='ground truth')

In [ ]:
# force colorings
plot_attraction_magnitude(Y)

In [ ]:
plot_attraction_directions(Y)
plot_polar_colormap(fig_scale=2)

In [ ]:
colorbar_kwargs = {'format':'%.0e', 'orientation':'vertical', 'pad':0.0, 'aspect':40}
plot_attraction_vectors(Y, fig_len=20, fig_width=20, ax_off=True, colorbar_kwargs=colorbar_kwargs)

In [ ]:
# attraction and repulsion cancel
plot_repulsion_vectors(Y, fig_len=20, fig_width=20, ax_off=True, colorbar_kwargs=colorbar_kwargs)

In [ ]:
# Let's also generate plots for some MNIST clusters.

X, lab = loadlocal_mnist_classes([1,5], n=None, seed=None)
X_red = pca(X, n_dims=50)

FRAC = 0.01
K=int(FRAC * X.shape[0])
%time Y = fast_tsne(X_red, early_exag_coeff=12, perplexity=-1, K=K, sigma=1e10, load_affinities='save')

In [ ]:
plot_blind(Y, suptitle='unlabeled embedding')
plot_ground_truth(Y, lab, cmap='rainbow', suptitle='ground truth')

In [ ]:
plot_attraction_magnitude(Y)
plot_attraction_directions(Y)

In [ ]:
colorbar_kwargs = {'format':'%.0e', 'orientation':'vertical', 'pad':0.0, 'aspect':40}
plot_attraction_vectors(Y, fig_len=20, fig_width=20, ax_off=True, colorbar_kwargs=colorbar_kwargs)

In [ ]:
# Examine MNIST images
attraction_directions = compute_attraction_directions(Y)
plot_reference_grid(Y, lab, attraction_directions, cmap='hsv', cell_width=10, cell_height=10)


In [ ]:
# plot mean image in each grid cell 
plot_mean_images(X, Y, (28,28), cell_width=10, cell_height=10, fig_scale=1, show_counts=True)